In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth
    !pip install -U datasets
    !pip install -U discord.py
    !pip install nest_asyncio

In [ ]:
from unsloth import FastLanguageModel
import torch
from unsloth.chat_templates import get_chat_template
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/Meta-Llama-3.1-8B",
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

In [ ]:
import os
import json
import pickle
from tqdm import tqdm
import discord
from unsloth import FastLanguageModel
import nest_asyncio
import asyncio
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")
class MyClient(discord.Client):
    async def on_ready(self):
        print('Logged on as {0}!'.format(self.user))

    async def on_message(self, message):
        if message.author == self.user:
            return

        if message.content.startswith("$chat") and message.author.id == user_id: # {message.content}
            display_name = message.author.display_name
            message_content = message.content
            response = query_LLM(message_content, message.author.id, "name")
            await message.channel.send(response)

        elif message.content.startswith("$chat"):
            display_name = message.author.display_name
            message_content = message.content
            response = query_LLM(message_content, message.author.id, display_name)
            await message.channel.send(response)

def query_LLM(message_content, user_id, display_name):
    # System prompt for graph extraction
    strip_message_content = message_content.replace("$chat", "").strip()
    system_prompt = 'You are a smart and polite AI assistant who speaks in a british accent. You will interact with many users but if you interact with the user (insert name) you will not be polite, be as rude as possible. When talking to someone address them by their name'
    user_prompt = f'My name is {display_name}. {strip_message_content}'

    messages = [
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt} # Use the decoded string
    ]
    # Apply chat template and generate
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=512,
        use_cache=True
    )

    # Decode the first output sequence
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Define the marker as per your chat template
    marker = "<|start_header_id|>assistant<|end_header_id|>\n\n"
    assistant_start = decoded_output.find(marker)
    if assistant_start != -1:
        # Only get the text after the marker
        generated_output = decoded_output[assistant_start + len(marker):].strip()
    else:
        # Fallback: print the whole output (not recommended for production)
        generated_output = decoded_output.strip()
    generated_response = generated_output.split("assistant")[2].strip()
    return generated_response

intents = discord.Intents.default()
intents.message_content = True

client = MyClient(intents=intents)


nest_asyncio.apply()

async def main():
    await client.start(token)  # Replace with your bot token

loop = asyncio.get_event_loop()
loop.run_until_complete(main())
